QD = Quick draw model from Kevin Mader

https://github.com/mjpyeon/wavenet-classifier/blob/master/WaveNetClassifier.py

Dataset is the raw RADAR data that has not been reshaped into chirps.

## Notebook setup

Allow editing of modules using editor (auto reloading)

In [0]:
# Needed to allow editing using PyCharm etc
%load_ext autoreload
%autoreload 2

The following cell is needed for compatibility when using both CoLab and Local Jupyter notebook. It sets the appropriate file path for the data and also installs local packages such as models and data_loading.

In [2]:
import os
path = os.getcwd()
if path == '/content':
    from google.colab import drive
    drive.mount('/content/gdrive')
    BASE_PATH = '/content/gdrive/My Drive/Level-4-Project/'
    !cd gdrive/My\ Drive/Level-4-Project/ && pip install --editable .
    os.chdir('gdrive/My Drive/Level-4-Project/')
    
elif path == 'D:\\Google Drive\\Level-4-Project\\notebooks\\wavenet':
    BASE_PATH = "D:/Google Drive/Level-4-Project/"
    
elif path == "/export/home/2192793m":
    BASE_PATH = "/export/home/2192793m/Level-4-Project/"
    
DATA_PATH = BASE_PATH + 'data/processed/wavenet/raw_reshaped/'
MODEL_PATH = BASE_PATH + 'models/wavenet/raw_reshaped/test_8/'
RESULTS_PATH = BASE_PATH + 'results/wavenet/raw_reshaped/'
    
from src.data import load_data
from src.visualization import multiple_plots, visualize, plot_confusion_matrix
from src.features import make_spectrograms, process_labels, make_directory
from src.models.dilated_CNN.wavenet_kevin_mader import WaveNetClassifier
from src.data_generator.data_generator import DataGenerator

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Obtaining file:///content/gdrive/My%20Drive/Level-4-Project
  Found existing installation: src 0.1.0
    Can't uninstall 'src'. No files were found to uninstall.
  Running setup.py develop for src


Using TensorFlow backend.


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import classification_report, confusion_matrix
from keras import metrics
from keras import optimizers
from keras.callbacks import History, ModelCheckpoint, CSVLogger
from keras.models import load_model

In [0]:
# Load in data dictionary.
# This does not load in any actual data,
# just the dictionary with the names of the files and their associated labels
with open(DATA_PATH + "index.pkl", "rb") as file:
    data = pickle.load(file)

In [0]:
# Remove user C as this user is reserved for the test set
# try:
#     del data["C"]
# except KeyError:
#     print ("Key 'C' not found")

In [0]:
def convert_label_to_int(label):
    if label == "walking":
        return 0
    if label == "pushing":
        return 1
    if label == "sitting":
        return 2
    if label == "pulling":
        return 3
    if label == "circling":
        return 4
    if label == "clapping":
        return 5
    if label == "bending":
        return 6

In [0]:
labels = {}
partition = {'train':[], 'validation':[]} # contains list of training and validation ID's
validation_user = "C"

for user_letter, actions in data.items():
    for action, results in actions.items():
        for result in results:
            for row in result:
                if user_letter == validation_user:
                    partition["validation"].append(row)
                    labels[row] = convert_label_to_int(action)

                else:
                    partition["train"].append(row)
                    labels[row] = convert_label_to_int(action)

In [0]:
target_names = ["walking", "pushing", "sitting", "pulling", "circling", "clapping", "bending"]
nb_classes = len(target_names)

In [0]:
data_shape = (3000, 128)
n_filters = 64
dilation_depth = 8
activation = 'softmax'
scale_ratio = 1
kernel_size = 2
pool_size_1 = 4
pool_size_2 = 8
batch_size = 16
epochs = 30

In [0]:
# Parameters
params = {'dim': (data_shape),
          'batch_size': batch_size,
          'n_classes': nb_classes,
          'data_directory': DATA_PATH}
# Generators
training_generator = DataGenerator(partition['train'], labels, **params, shuffle=True)
validation_generator = DataGenerator(partition['validation'], labels, **params, shuffle=False, data_type="Validation")

In [0]:
wnc = WaveNetClassifier(data_shape, (7,), kernel_size = kernel_size,
                        dilation_depth = dilation_depth, n_filters = n_filters,
                        pool_size_1=pool_size_1, pool_size_2=pool_size_2)

In [0]:
model = wnc.get_model()

In [0]:
load_weights = False
weights_path = MODEL_PATH + "epoch-07-val_acc-0.14.hdf5"

start_epoch = 0
if load_weights:
    model = load_model(weights_path)
#     model.load_weights(weights_path)
    last_epoch = weights_path.split("-")[-3]
    start_epoch = int(last_epoch)

In [0]:
if not load_weights:
#     adam = optimizers.Adam(lr=0.01)
    model.compile("adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
checkpoint = ModelCheckpoint(MODEL_PATH + "epoch-{epoch:02d}-val_acc-{val_acc:.2f}.hdf5",
                             monitor='val_acc', verbose=0, save_best_only=False,
                             save_weights_only=False, mode='auto', period=1)

csv_logger = CSVLogger(RESULTS_PATH + "test_8.csv", append=True)
callbacks_list = [checkpoint, csv_logger]

In [16]:
# Train model on dataset
history = model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=9,
                    epochs=epochs,
                    callbacks=callbacks_list,
                    initial_epoch=start_epoch)

Epoch 1/30
2315/2315 [==============================] - 4664s 2s/step - loss: 1.9488 - acc: 0.1501 - val_loss: 1.9478 - val_acc: 0.1431

Epoch 2/30
2315/2315 [==============================] - 4615s 2s/step - loss: 1.9463 - acc: 0.1523 - val_loss: 1.9471 - val_acc: 0.1431
Epoch 3/30
2315/2315 [==============================] - 4596s 2s/step - loss: 1.9463 - acc: 0.1522 - val_loss: 1.9515 - val_acc: 0.1431
Epoch 4/30
2314/2315 [============================>.] - ETA: 1s - loss: 1.9465 - acc: 0.1514

Process ForkPoolWorker-80:
Process ForkPoolWorker-75:
Process ForkPoolWorker-73:
Process ForkPoolWorker-81:
Process ForkPoolWorker-77:
Process ForkPoolWorker-78:
Process ForkPoolWorker-74:
Process ForkPoolWorker-76:
Process ForkPoolWorker-79:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-72:
Process ForkPoolWorker-68:
Process ForkPoolWorker-67:
Process ForkPool

KeyboardInterrupt: ignored

In [0]:
visualize.plot_train_test_acc(history=history)

In [0]:
visualize.plot_train_test_loss(history=history)

The following graph suggests that the model began to overfit after the 6th (16th total) epoch